<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Imports</a></span></li><li><span><a href="#First-steps" data-toc-modified-id="First-steps-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>First steps</a></span></li><li><span><a href="#Web-Scraping-functions" data-toc-modified-id="Web-Scraping-functions-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Web Scraping functions</a></span></li><li><span><a href="#Running-Scraping" data-toc-modified-id="Running-Scraping-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Running Scraping</a></span></li></ul></div>

# Scraping Yelp with Beautiful Soup
We will scrape top restaurants reviews in New York

## Imports

In [2]:
from bs4 import BeautifulSoup
import requests as rq
import pandas as pd
import re
import time
from tqdm import tqdm, tqdm_notebook
tqdm_notebook().pandas

<bound method tqdm.pandas of <class 'tqdm._tqdm_notebook.tqdm_notebook'>>

## First steps
Creating a BeautifulSoup object with the webpage we want, here it's the default page when we specify restaurants in New York

In [3]:
html = rq.get('https://www.yelp.com/search?find_desc=Restaurants&find_loc=New+York,+NY,+%C3%89tats-Unis').text
soup = BeautifulSoup(html,"lxml")

An empty DataFrame that will store all of our data

In [4]:
restaurants = pd.DataFrame()

## Web Scraping functions

Function to get Name and Address of each restaurants, and call the get_reviews() function

In [14]:
def fill_restaurants(restaurants, entries):
    
    new_restaurants1 = pd.DataFrame()
    for s,i in tqdm(zip(entries,range(len(entries)))):
        try:      
            link = s.find('a', attrs={'class':'biz-name js-analytics-click'})['href']
            reviews = get_reviews(link)
            for t in range(reviews.shape[0]):
                reviews.loc[t,'name']=s.find('a',attrs={'class':'biz-name'}).get_text().strip()
                reviews.loc[t,'adress']=s.find('address').contents[0].strip()
            new_restaurants1 = pd.concat([new_restaurants1,reviews],ignore_index=True)
        except:
            pass
    restaurants = pd.concat([restaurants,new_restaurants1],ignore_index=True)
    time.sleep(5)
    
    return restaurants

Function to get the reviews from a restaurant

In [6]:
def get_reviews(link):
    
    text = []
    stars= []
    html_link = rq.get('https://www.yelp.com'+link).text
    soup_link = BeautifulSoup(html_link,"lxml")
    
    number_of_pages = soup_link.find('div', attrs={'class':'page-of-pages arrange_unit arrange_unit--fill'}).get_text().strip()
    number_of_pages = int(number_of_pages.split('of ')[1])
    
    if(number_of_pages<2):#get the restaurants with a min of 20 reviews
        return None
    if(number_of_pages>7): #limit the number of reviews to 140
        number_of_pages = 7

    for i in range(number_of_pages):
        reviews_list = soup_link.find('ul', attrs={'class':'ylist ylist-bordered reviews'})
        for s,n in zip(reviews_list,range(len(reviews_list))):
            try:
                large_star = s.find('div',attrs={'biz-rating biz-rating-large clearfix'})
                stars.append(large_star.find('img')['alt'][:3])
                text.append(s.find('p').get_text(strip=True, separator=' '))
            except:
                pass
            
        if (i!=number_of_pages-1):
            link = soup_link.find('a', attrs={'u-decoration-none next pagination-links_anchor'})['href']
            html_link = rq.get(link).text
            soup_link = BeautifulSoup(html_link,"lxml")
        time.sleep(5)
        
    reviews = pd.DataFrame({'review':text,'stars':stars})
    return reviews

## Running Scraping

First initialization

In [15]:
entries = soup.find_all('div',attrs={'class':'search-result'})
restaurants = fill_restaurants(restaurants, entries)

0it [00:00, ?it/s]


Then looping through the pages (there are 33 pages for New York because page 34 doesn't work).

Yelp have blocked me after 1h30 hours (including previous cell) which retrieved me 13 000 reviews.

In [87]:
number_of_pages = 1 #out of 33
for i in range(number_of_pages):
    next_page_link = soup.find('a',attrs={'class':'u-decoration-none next pagination-links_anchor'})['href']
    html = rq.get('https://www.yelp.com'+next_page_link).text
    soup = BeautifulSoup(html,"lxml")
    entries = soup.find_all('div',attrs={'class':'search-result'})
    restaurants = fill_restaurants(restaurants,entries)

31it [21:50, 42.27s/it]


In [93]:
restaurants.to_csv('restaurants_ny.csv',index=False)